
<h1>petersen2smiley</h1>

<br/><br/>
    
Since we reached 100% on the test set, we wanted to come up with a creative idea. The plan was as follows:


1. Extract faces of Prof. Petersen from the frames of the recordings of lectures with the FaceNet 
   network (https://github.com/timesler/facenet-pytorch).
2. Convert the numpy array into a format we can train our CycleGAN in the next step which are 
   jpg images in RGB.
3. Train a petersen2smiley CycleGAN (https://github.com/LynnHo/CycleGAN-Tensorflow-2) between Prof. Petersen 
   and the Facial Expressions.
4. After doing the inference we still have the annotated data - now having a face of Prof. Petersen 
   corresponding to a facial expression. 
5. We would now use the Autokeras (https://autokeras.com/) library to train the classifier in the usual way.

CycleGAN doesn't need a correspondence between the source and target domain, which very good fits into
our idea. The idea of CycleGAN is to learn a mapping G: X -> Y where the distribution of images betwen Y and 
G(X) is indistinguishable. 

Problems that arose with this type of generative adversarial network is the very long training time to even
get reasonable results (we trained on one middle class GPU). 

We thus just present some of the more interesting results of the CycleGAN.



### Imports

In [1]:
import numpy as np
from facenet_pytorch import MTCNN
import cv2
from PIL import Image
from tqdm.notebook import tqdm

smileys = np.load("data_train_Facial.npy", allow_pickle = True)
labels = np.loadtxt("true_labels_Facial_train.csv", delimiter = ",")
smiley_count = smileys.shape[0]
image_size = smileys.shape[1]

video = cv2.VideoCapture("lectures.mp4")
video_length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

### Extract Faces from Lectures

In [3]:
# create face detector
mtcnn = MTCNN(image_size = image_size, select_largest = False, post_process = False)

# loop through video frames
for frame_number in tqdm(range(video_length)):
    
    # load frame
    success, frame = video.read()
    if not success:
        continue
        
    # convert to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # save face
    mtcnn(frame, save_path = f"CycleGAN\\datasets\\petersen2smiley\\trainB\\face_{frame_number:05d}.jpg")

# save last face to test folder
mtcnn(frame, save_path = "CycleGAN\\datasets\\petersen2smiley\\trainB\\face_test.jpg")

### Save Smileys as PNGs

In [4]:
for index in tqdm(range(smiley_count)):
    
    smiley = np.uint8(smileys[index])
    
    # create empty image
    image = Image.new(mode = "RGB", size = smiley.shape)
    pixels = image.load()
    
    # loop through pixels
    for i in range(image.size[0]):
        for j in range(image.size[1]):
            
            # set pixel to white
            if smiley[i,j] == 0:
                pixels[j,i] = (255, 255, 255)
                
            # set pixel to yellow
            elif smiley[i,j] == 1:
                pixels[j,i] = (255, 255, 0)
                
            # else pixels are black
        
    # save smiley image
    image.save(f"CycleGAN\\datasets\\petersen2smiley\\trainA\\smiley_{index:05d}.jpg")

# save last smiley to test folder
image.save("CycleGAN\\datasets\\petersen2smiley\\testA\\smiley_test.jpg")

### Train CycleGAN

zip the whole folder and train cyclegan with these images

In [ ]:
# python train.py --dataset petersen2smiley

In [ ]:
from IPython import display
display.Image("./examples/1.png")

In [ ]:
display.Image("./examples/2.png")

In [ ]:
display.Image("./examples/3.png")

In [ ]:
display.Image("./examples/4.png")

In [ ]:
display.Image("./examples/5.png")

In [ ]:
display.Image("./examples/6.png")

In [ ]:
display.Image("./examples/7.png")

In [ ]:
display.Image("./examples/8.png")

### Use Expression Classifier